In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Mon May 16 02:33:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 55%   59C    P8    43W / 260W |   4238MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 37%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_de = db.patent_de
    db_us = db.patent_us
    db_cn = db.patent_cn
    # db_test1 = db.test_1
    # db_test2 = db.test_2
    # db_rm_us = db.rm_patent_us
    # db_rm_cn = db.rm_patent_cn
    # db_rm_de = db.rm_patent_de
    # db_rm_cn_5 = db.rm_patent_cn_5
    # db_rm_cn_10 = db.rm_patent_cn_10
    # db_rm_us_5 = db.rm_patent_us_5
    # db_rm_us_10 = db.rm_patent_us_10
    # db_rm_de_5 = db.rm_patent_de_5
    # db_rm_de_10 = db.rm_patent_de_10
except errors.ConnectionFailure as err:
    print(err)

In [6]:
db_de

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'patent_de')

## CN-US

### 留下CN中priority和US重複的

In [7]:
dbcn = pd.DataFrame(list(db_cn.find()))

In [8]:
dbcn.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,61fd2531f4d57576a5021667,CN103890730B,"[普拉蒂普·米斯拉, 阿尔潘·帕尔, 巴拉穆拉利达尔·普鲁肖撒曼, 集拉布拉塔·包米克, 迪...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20170711,N/A,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",本文公开了用于车辆遥测应用的智能开发、部署和管理的计算平台。此外，本发明提供了一种允许在基于...,用于传感器驱动的车辆遥测应用和服务的开发和部署的计算平台,用于传感器驱动的车辆遥测应用和服务的开发和部署的计算 平台技术领域[0001]本发明通常涉及...,[1.一种智能交通系统，包括；(a)基础设施平台，其包括多个硬件和软件组件、基础设施服务、应...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'CN', 'patentNumber': '163364', 'kind...","[{'pto': 'US', 'patentNumber': '527662', 'kind...",None,"[{'pto': 'US', 'patentNumber': '9932033', 'kin...","[{'publication': 'CN103890730B', 'time': '2017...",IN2651/MUM/2011
1,61fd25e1f4d57576a5021669,CN103492389B,"[M·阿尔姆施泰特尔, M·托尔曼, A·特雷姆尔, N·特劳贝]",[Origenis GmbH],[Origenis GmbH],20120423,20160914,N/A,"[C07D487/04, A61K31/519, A61P1/00, A61P11/00, ...",本发明涉及式(I)的新化合物，其能够抑制一种或多种激酶，尤其抑制SYK(脾酪氨酸激酶)、LR...,"用作激酶抑制剂的吡唑并[4,3-d]嘧啶",技术领域[0001 ]本发明涉及能够抑制一种或多种激酶，尤其SYK(脾酪氨酸激酶）、LRRK...,[1.式(I)的化合物或其药学上可接受的盐其中A为NH;Ri为任选地被取代的杂烷基、C6-1...,"[US, EP, JP, KR, CN, AP, AU, BR, CA, CL, CO, C...","[{'pto': 'WO', 'patentNumber': '1998029413', '...","[{'pto': 'US', 'patentNumber': '446986', 'kind...",None,"[{'pto': 'KR', 'patentNumber': '2014002097', '...","[{'publication': 'CN103492389B', 'time': '2016...",US201161517582P
2,61fd25e5f4d57576a502166b,CN102393961A,"[王滨海, 张晶晶, 王骞, 刘俍, 王万国]",[State Grid Intelligent Technology Co Ltd],[Electric Power Research Institute of State Gr...,20110629,20120328,N/A,None,本发明公开了一种在复杂自然环境背景下，实现了基于计算机视觉的飞行器输电线路实时检测跟踪方法。...,基于计算机视觉的飞行器输电线路实时检测跟踪方法,基于计算机视觉的飞行器输电线路实时检测跟踪方法技术领域[0001]\t本发明涉及一种图像处理...,[1.一种基于计算机视觉的飞行器输电线路实时检测跟踪方法，其特征是，首先使用抗噪声的梯度图像...,[CN],None,None,"[{'pto': 'CN', 'patentNumber': '10314993', 'ki...",None,"[{'publication': 'CN102393961A', 'time': '2012...",CN201110179733.2A
3,61fd26a7f4d57576a502166d,CN103155174B,"[迈克尔·尤金·杨, 阿琼·丹尼尔·瑞尼瓦斯, 马修·R.·鲁滨逊, 亚历山大·周·米塔尔]",[Chen Hung Technology Holdings Limited],[Chen Hung Technology Holdings Ltd],20110808,20170623,N/A,"[H05K1/09, H01L31/048, G06F3/0445, H01L31/0481...",将活性或功能性添加剂嵌入到用作包括太阳能装置、智能窗、显示器等的各种电子或光电装置的组件的主...,具有表面嵌入的添加剂的装置组件和相关的制造方法,具有表面嵌入的添加剂的装置组件和相关的制造方法[0001] 相关申请的交叉引用[0002] ...,[1.一种太阳能装置，包括：前覆盖层；后覆盖层；光敏层，所述光敏层被设置在所述前覆盖层与所述...,"[US, EP, JP, KR, CN, AU, CA, WO]","[{'pto': 'CN', 'patentNumber': '10108814', 'ki...","[{'pto': 'US', 'patentNumber': '375128', 'kind...",None,"[{'pto': 'US', 'patentNumber': '9530940', 'kin...","[{'publication': 'CN103155174B', 'time': '2017...",US37168810P
4,61fd26c2f4d57576a502166f,CN103958612B,"[服部公彦, 玉井晃义, 歌崎宪, 歌崎宪一]",[Toray Industries Inc],[Toray Industries Inc],20121119,20160824,N/A,"[C08K3/04, C08L101/00, B29B7/90, B29B9/06, B29...",本发明涉及一种碳纤维增强热塑性树脂组合物及其成型品，所述成型品为使用含有(A)热塑性树脂、(...,碳纤维增强热塑性树脂组合物、该组合物的粒料和成型品,技术领域[0001] 本发明的课题在于提供一种机械特性、表面外观等优异、特别是弯曲弹性模量、...,[1.一种碳纤维增强热塑性树脂组合物，为含有(A)热塑性树脂、（B)碳纤维和(C)钛化合物的...,"[US, EP, JP, KR, CN, TW, WO]","[{'pto': 'CN', 'patentNumber': '10179613', 'ki...","[{'pto': 'JP', 'patentNumber': 'S629541', 'kin...",None,"[{'pto': 'JP', 'patentNumber': '5987335', 'kin...","[{'publication': 'CN103958612B', 'time': '2016...",JP2011-259853


In [9]:
dbus = pd.DataFrame(list(db_us.find()))

In [10]:
dbus.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,62009a5e7b73c16636b31082,US9990182B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20180605,20180516,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A computing platform for intelligent developme...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '527662', 'kind...","[{'pto': 'JP', 'patentNumber': '200126069', 'k...","[{'pto': 'US', 'patentNumber': '10965787', 'ki...","[{'pto': 'US', 'patentNumber': '9932033', 'kin...","[{'publication': 'US9990182B2', 'time': '2018-...",IN2651/MUM/2011
1,62009a807b73c16636b31084,US20190027414A1,"[Senthil Kumar Ramadas, Saravanan Shanmugavel]","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...",20180924,20190124,N/A,"[B32B7/02, H01L23/29, B82Y30/00, C23C28/00, C2...","Disclosed is an encapsulation barrier stack, c...",Encapsulationbarrierstack,The present application is a divisional applic...,[1.Amethodofmanufacturinganencapsulationbarrie...,"[US, EP, JP, KR, CN, AU, CA, DK, MX, MY, RU, S...","[{'pto': 'US', 'patentNumber': '20100089636', ...","[{'pto': 'US', 'patentNumber': '560778', 'kind...","[{'pto': 'US', 'patentNumber': '10854853', 'ki...","[{'pto': 'US', 'patentNumber': '20150314941', ...","[{'publication': 'US20190027414A1', 'time': '2...",US201161550764P
2,62009aea7b73c16636b31086,US10216485B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20190226,20190206,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A method and system for real-time analytics of...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '20020128751', ...","[{'pto': 'US', 'patentNumber': '7779383', 'kin...","[{'pto': 'US', 'patentNumber': '20180373418', ...","[{'pto': 'US', 'patentNumber': '8250213', 'kin...","[{'publication': 'US10216485B2', 'time': '2019...",IN2651/MUM/2011
3,62009b557b73c16636b31088,US20140310243A1,"[Steven James McGee, Tammy Lee McGee]",[],[Mr. Steven James McGee],20120813,20141016,N/A,"[G06F17/30575, G06F16/27, G06Q10/101, G06Q10/1...","Systemic, adaptive procedural template compris...",Heartbeaconcycle,Related applications by Applicant Steven J. Mc...,"[1.Asystemic,adaptiveproceduraltemplatemethodu...",[US],"[{'pto': 'US', 'patentNumber': '20060217138', ...",None,"[{'pto': 'US', 'patentNumber': '20130086129', ...",None,"[{'publication': 'US20140310243A1', 'time': '2...","US12/856,715"
4,62009b717b73c16636b3108a,US8965375B2,"[Juergen Michel, Dirk Rose]",[Nokia Solutions and Networks Oy],[Nokia Solutions and Networks Oy],20110504,20150224,20150204,"[H04W36/04, H04W24/10, H04W52/0206, H04W52/023...",There are provided measures for a pathloss-bas...,Pathlossbasedaccessnodewake-upcontrol,The present invention relates to a pathloss-ba...,[1.Amethodcomprisingretrievinganaveragehandove...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '6498932', 'kin...",None,"[{'pto': 'US', 'patentNumber': '20150043538', ...","[{'pto': 'US', 'patentNumber': '9203489', 'kin...","[{'publication': 'US8965375B2', 'time': '2015-...",PCT/EP2011/057133


In [15]:
p_ls = list(set(pd.merge(dbcn,dbus,on=['priority'],how='inner')['priority']))

In [18]:
result_cn = dbcn[dbcn.priority.isin(p_ls)]

In [19]:
result_cn

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,61fd2531f4d57576a5021667,CN103890730B,"[普拉蒂普·米斯拉, 阿尔潘·帕尔, 巴拉穆拉利达尔·普鲁肖撒曼, 集拉布拉塔·包米克, 迪...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20170711,N/A,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",本文公开了用于车辆遥测应用的智能开发、部署和管理的计算平台。此外，本发明提供了一种允许在基于...,用于传感器驱动的车辆遥测应用和服务的开发和部署的计算平台,用于传感器驱动的车辆遥测应用和服务的开发和部署的计算 平台技术领域[0001]本发明通常涉及...,[1.一种智能交通系统，包括；(a)基础设施平台，其包括多个硬件和软件组件、基础设施服务、应...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'CN', 'patentNumber': '163364', 'kind...","[{'pto': 'US', 'patentNumber': '527662', 'kind...",None,"[{'pto': 'US', 'patentNumber': '9932033', 'kin...","[{'publication': 'CN103890730B', 'time': '2017...",IN2651/MUM/2011
10,61fd273cf4d57576a502167b,CN106851751B,"[闵粲浩, 郑丁寿, 金东煜, 郑景仁, 权种炯, 林彩权, 全泳炫]",[Samsung Electronics Co Ltd],[Samsung Electronics Co Ltd],20110427,20210101,N/A,"[H04W36/0055, H04W36/0058, H04W36/0083, H04W36...",提供了一种移动通信系统中用于提供切换支持信息的装置和方法。一种移动通信系统中移动站(MS)确...,在移动通信系统中提供切换支持信息的装置和方法,本申请是申请日为2011年04月27日、申请号为201180021422.1、发明名称为“在...,[1.一种由移动通信系统中的基站BS执行的方法，所述方法包括：生成包含指示针对至少一个第一相...,"[US, EP, JP, KR, CN, WO]","[{'pto': 'CN', 'patentNumber': '167124', 'kind...","[{'pto': 'KR', 'patentNumber': '2004008154', '...",None,"[{'pto': 'JP', 'patentNumber': '5174580', 'kin...","[{'publication': 'CN106851751B', 'time': '2021...",KR1020100038886A
23,61fd2962f4d57576a5021695,CN106131926B,"[本哥特·林多夫, 拉尔斯·林德布姆, 斯蒂芬·帕克维尔]",[Telefonaktiebolaget LM Ericsson AB],[Telefonaktiebolaget LM Ericsson AB],20110614,20210202,N/A,"[H04W48/16, H04W24/10, H04W8/005, H04W88/08]",一种扩展小区搜索过程，使异类网络中操作的移动终端能够实现更加包容性的测量报告。移动终端可以被...,异类网络中的小区搜索与测量,本申请是2012年12月10日提交的、申请号为201180028532.0、发明名称为“异类...,[1.一种由移动终端实现的小区搜索方法，所述方法包括：基于从基站接收的控制信息，在通常小区搜...,"[US, EP, CN, CA, MY, SG, WO]","[{'pto': 'CN', 'patentNumber': '10150733', 'ki...","[{'pto': 'FI', 'patentNumber': '10212', 'kindc...",None,"[{'pto': 'US', 'patentNumber': '8649326', 'kin...","[{'publication': 'CN106131926B', 'time': '2021...",US35490010P
27,61fd29fdf4d57576a502169d,CN103748562B,"[M·特罗布夫, K·蒂鲁瓦卢尔, C·普鲁维, C·罗文, D·格劳罗克, J·内耶罗, ...",[Intel Corp],[Intel Corp],20101223,20190329,N/A,"[G06F11/267, G06F11/2733, G06F11/36, G06F11/22...",本文描述了用于提供测试、验证和调试架构的装置和方法。在目标级或基础级，硬件钩子(用于测试的设...,测试、验证和调试架构,技术领域本发明涉及计算机系统的领域，并且特别涉及为计算机系统提供测试和调试基础设施。背景技术...,[1.一种用于测试的装置，包括：集成电路，包括适合于捕获与所述集成电路相关联的测试/踪迹信息...,"[US, JP, KR, CN, DE, GB, WO]","[{'pto': 'CN', 'patentNumber': '119441', 'kind...","[{'pto': 'JP', 'patentNumber': 'H0831559', 'ki...",None,"[{'pto': 'US', 'patentNumber': '7895565', 'kin...","[{'publication': 'CN103748562B', 'time': '2019...",PCT/US2010/061995
66,61fd2d00f4d57576a50216eb,CN104012149B,[让-菲利普·瓦瑟尔],[Cisco Technology Inc],[Cisco Technology Inc],20121220,20180424,N/A,"[H04L45/42, H04L45/64, H04W40/24, H04L41/00, H...",在一个实施例中，网络架构包括简约连接对象(MCO)、分布式智能代理(DIA)、以及中央智能控...,用于简约连接对象的网络架构,用于简约连接对象的网络架构技术领域[0001] 本公开一般地涉及计算机网络。背景技术[000...,[1.一种用于简约连接对象的系统，包括：多个简约连接对象(MCO)，每个MCO具有刚好足以执...,"[US, EP, JP, CN, CA, WO]",None,"[{'pto': 'US', 'patentNumber': '6990086', 'kin...",None,"[{'pto': 'JP', 'patentNumber': '2869025', 'kin...","[{'publication': 'CN104012149B', 'time': '2018...","US13/331,776"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47225,620072bff4d57576a5038759,CN111600698A,"[张栩瑞, 丁裕源]",[Advanced Semiconductor Engineering Inc],[Advanced Semiconductor Engineering Inc],20190401,20200828,N/A,"[H04W52/0229, H04L7/0008, H04L5/16, H04W4/44, ...",本公开涉及一种通信系统。所述通信系统包含发射模块。所述发射模块被配置成将至少一个前同步码发送...,通信系统,技术领域本公开涉及一种通信系统和用于操作所述通信系统的方法，且更确切地说，涉及一种系统和用于...,[1.一种通信系统，其包括：发射模块，其被配置成将至少一个前同步码发送到接收模块，所述接收模...,"[US, CN]",None,None,None,None,"[{'publication': 'KR100729169B1', 'time': '200...","US16/280,862"
47226,620072c0f4d57576a503875b,CN111903077A,"[金泰亨, 金泳范, 崔承勋]",[Samsung Electronics Co Lt

In [20]:
result_us = dbus[dbus.priority.isin(p_ls)]
result_us

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,62009a5e7b73c16636b31082,US9990182B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20180605,20180516,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A computing platform for intelligent developme...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '527662', 'kind...","[{'pto': 'JP', 'patentNumber': '200126069', 'k...","[{'pto': 'US', 'patentNumber': '10965787', 'ki...","[{'pto': 'US', 'patentNumber': '9932033', 'kin...","[{'publication': 'US9990182B2', 'time': '2018-...",IN2651/MUM/2011
2,62009aea7b73c16636b31086,US10216485B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20190226,20190206,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A method and system for real-time analytics of...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '20020128751', ...","[{'pto': 'US', 'patentNumber': '7779383', 'kin...","[{'pto': 'US', 'patentNumber': '20180373418', ...","[{'pto': 'US', 'patentNumber': '8250213', 'kin...","[{'publication': 'US10216485B2', 'time': '2019...",IN2651/MUM/2011
6,62009bab7b73c16636b3108e,US10602420B2,"[Chan-Ho Min, Jung-Soo Jung, Dong-Wook Kim, Ky...",[Samsung Electronics Co Ltd],[Samsung Electronics Co Ltd],20160127,20200324,20200304,"[H04W36/32, H04W36/0055, H04W36/0058, H04W36/0...",An apparatus and method for providing handover...,Apparatusandmethodforprovidinghandoversupporti...,This application is a continuation application...,[1.Amethodforoperatingaservingbasestation(BS)i...,"[US, EP, JP, KR, CN, WO]","[{'pto': 'KR', 'patentNumber': '2004008154', '...","[{'pto': 'WO', 'patentNumber': '2008151549', '...",None,"[{'pto': 'JP', 'patentNumber': '5174580', 'kin...","[{'publication': 'US10966129B2', 'time': '2021...",KR1020100038886A
12,62009c0f7b73c16636b3109a,US20130223235A1,"[Rose Qingyang Hu, Zhijun Cai, Chandra Sekhar ...",[BlackBerry Ltd],[Research in Motion Ltd],20100714,20130829,N/A,"[H04W36/30, H04W48/20, H04W36/04, H04W84/045]",A UE comprising a processor configured to perf...,IdleModeHybridMobilityProcedureinaHeterogeneou...,This application is a filing under 35 U.S.C. 3...,[1.AUEcomprising:aprocessorconfiguredtoperform...,"[US, EP, JP, KR, CN, CA, IN, TW, WO]","[{'pto': 'US', 'patentNumber': '20070123252', ...","[{'pto': 'GB', 'patentNumber': '0413534', 'kin...","[{'pto': 'US', 'patentNumber': '20120276933', ...","[{'pto': 'KR', 'patentNumber': '2012003490', '...","[{'publication': 'US20130223235A1', 'time': '2...",PCT/US2010/042018
33,62009ffc7b73c16636b310c4,US9451555B2,[Eldad M. Zeira],[InterDigital Patent Holdings Inc],[InterDigital Patent Holdings Inc],20150604,20160920,20160831,"[H04W52/242, H04L5/0007, H04W52/16, H04W52/383...",Method and apparatus for controlling a transmi...,Methodforcontrollingtransmitpowerofamobilestation,This application is a continuation of U.S. pat...,[1.Awirelesstransmitreceiveunit(WTRU)comprisin...,"[US, EP, CN, TW, WO]","[{'pto': 'US', 'patentNumber': '20050208961', ...","[{'pto': 'JP', 'patentNumber': '3286247', 'kin...","[{'pto': 'US', 'patentNumber': '20170332270', ...","[{'pto': 'KR', 'patentNumber': '2013002186', '...","[{'publication': 'US9832738B2', 'time': '2017-...",US201161496158P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45084,620b2c53e0ec9a30af2b76b9,US20210373337A1,"[J

In [21]:
# 存進MongoDB
db.p_patent_cn.insert_many(result_cn.to_dict('records'))

In [22]:
# 存進MongoDB
db.p_patent_us.insert_many(result_us.to_dict('records'))